In [9]:
import ast
import os
import random

import pandas as pd
import matplotlib.pyplot as plt
import sklearn.linear_model
import sklearn.metrics
import sklearn.model_selection
import autogluon.tabular
from IPython.display import display, HTML

DIRECTORY_DATA = 'data/model-autogluon'
os.makedirs(DIRECTORY_DATA, exist_ok=True)

COLUMNS_NONSPLIT = [
    'Scenario ID'
]
COLUMNS_INPUT = [
    'Vehicle ID', 
    # 'Linearization C',  
    *[f'Linearization D{i}' for i in range(4)],  
    'Coordination strategy', 
]
COLUMNS_OUTPUT = [
    'traveled total, m', 
    'No. of completed missions',
    'No. of collisions',
    'No. of near-misses',
]

In [12]:
runnames = [
    '20241203_170129_all600',
    '20241213_104400_racing',
    '20241214_122216_racing_passhum',
]
df_all = pd.concat(
    [pd.read_csv(f'data/{runname}/df_all.csv') 
     for runname in runnames],
    keys=runnames,
    names=['runname', 'row']
)
df_all.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 7176 entries, ('20241203_170129_all600', 0) to ('20241214_122216_racing_passhum', 2383)
Data columns (total 81 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Unnamed: 0                         2400 non-null   float64
 1   i_map                              7176 non-null   int64  
 2   are_bridges                        7176 non-null   bool   
 3   Positions variant                  7176 non-null   int64  
 4   configuration                      7176 non-null   object 
 5   Coordination strategy              7176 non-null   object 
 6   probabilityForcingForHuman         7176 non-null   float64
 7   filename_screenshot                7176 non-null   object 
 8   Date                               7176 non-null   object 
 9   Scenario ID                        7176 non-null   object 
 10  Vehicle ID                         7176 non-null   int64

In [13]:
df_inout = df_all[COLUMNS_NONSPLIT + COLUMNS_INPUT + COLUMNS_OUTPUT]
df_inout

Scenario ID  \
runname                        row                                                                                                                                                                      
20241203_170129_all600         0                         map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, stops, seed 1, probabilityForcingForHuman 1; AUTOMATED_FIRST   
                               1          map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, change of priorities, seed 1, probabilityForcingForHuman 1; AUTOMATED_FIRST   
                               2                      map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, baseline, seed 1, probabilityForcingForHuman 0; AUTOMATED_FIRST   
                               3                         map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, stops, seed 1, probabilityForcingForHuman 1; AUTOMATED_FIRST   
                               4          map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, change of priorities, seed 1, probabilityForcingForHuman 1; AUTOMATED_FIRST   
...                                                                                                                                                                                               ...   
20241214_122216_racing_passhum 2379  map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario10-10.json, change of priorities, seed 1, probabilityForcingForHuman 1; AUTOMATED_FIRST   
                               2380                 map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario10-10.json, stops, seed 1, probabilityForcingForHuman 1; AUTOMATED_FIRST   
                               2381              map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario10-10.json, baseline, seed 1, probabilityForcingForHuman 0; AUTOMATED_FIRST   
                               2382  map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario10-10.json, change of priorities, seed 1, probabilityForcingForHuman 1; AUTOMATED_FIRST   
                               2383                 map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario10-10.json, stops, seed 1, probabilityForcingForHuman 1; AUTOMATED_FIRST   

                                     Vehicle ID  \
runname                        row                
20241203_170129_all600         0              0   
                               1              0   
                               2              0   
                               3              1   
                               4              1   
...                                         ...   
20241214_122216_racing_passhum 2379           2   
                               2380           2   
                               2381           3   
                               2382           3   
                               2383           3   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             Linearization C  \
runname                        row                                                                                                                                                                                                                                          

In [14]:
def parse_tuple_string(tuple_string):
    if pd.isna(tuple_string):
        return None
    return ast.literal_eval(tuple_string)
    

def preprocess_inout(df_inout):
    df_inout = df_inout.copy()
    
    # Find the column that starts with 'Linearization'
    linearization_columns = [col for col in df_inout.columns if col.startswith('Linearization')]
    for col in linearization_columns:
        df_inout[col] = df_inout[col].apply(parse_tuple_string)
    
    # Create DataFrame with all preprocessed features
    df_preprocessed = pd.DataFrame({
        **{col: df_inout[col].values for col in COLUMNS_NONSPLIT},
        
        'Coordination strategy': df_inout['Coordination strategy'].astype('category').cat.codes.values,
        'Vehicle ID': df_inout['Vehicle ID'].values,
        **{f'{col}#{i}': df_inout[col].apply(lambda x: -1 if x is None else x[i])
           for col in linearization_columns
           for i in range(len(df_inout[col].dropna().iloc[0]))},

        **{col: df_inout[col].values for col in COLUMNS_OUTPUT},
    })
    
    return df_preprocessed
    

df_preprocessed = preprocess_inout(df_inout)
df_preprocessed

Scenario ID  \
runname                        row                                                                                                                                                                      
20241203_170129_all600         0                         map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, stops, seed 1, probabilityForcingForHuman 1; AUTOMATED_FIRST   
                               1          map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, change of priorities, seed 1, probabilityForcingForHuman 1; AUTOMATED_FIRST   
                               2                      map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, baseline, seed 1, probabilityForcingForHuman 0; AUTOMATED_FIRST   
                               3                         map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, stops, seed 1, probabilityForcingForHuman 1; AUTOMATED_FIRST   
                               4          map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, change of priorities, seed 1, probabilityForcingForHuman 1; AUTOMATED_FIRST   
...                                                                                                                                                                                               ...   
20241214_122216_racing_passhum 2379  map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario10-10.json, change of priorities, seed 1, probabilityForcingForHuman 1; AUTOMATED_FIRST   
                               2380                 map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario10-10.json, stops, seed 1, probabilityForcingForHuman 1; AUTOMATED_FIRST   
                               2381              map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario10-10.json, baseline, seed 1, probabilityForcingForHuman 0; AUTOMATED_FIRST   
                               2382  map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario10-10.json, change of priorities, seed 1, probabilityForcingForHuman 1; AUTOMATED_FIRST   
                               2383                 map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario10-10.json, stops, seed 1, probabilityForcingForHuman 1; AUTOMATED_FIRST   

                                     Coordination strategy  Vehicle ID  \
runname                        row                                       
20241203_170129_all600         0                         2           0   
                               1                         1           0   
                               2                         0           0   
                               3                         2           1   
                               4                         1           1   
...                                                    ...         ...   
20241214_122216_racing_passhum 2379                      1           2   
                               2380                      2           2   
                               2381                      0           3   
                               2382                      1           3   
                               2383                      2           3   

                                     Linearization C#0  Linearization C#1  \
runname                        row                                          
20241203_170129_all600         0                   0.0                0.0   
                               1                   0.0                0.0   
                               2                   0.0                0.0   
                               3                   0.0                0.0   
                               4                   0.0                0.0   
...                                                ...                

In [15]:
def show(obj, title=None):
    if title is not None:
        display(HTML(f"<h3>{title}</h3>"))
    display(obj)


def shuffle_df(df):
    # Shuffle by Scenario ID
    unique_scenarios = df['Scenario ID'].unique()  # Get unique Scenario IDs
    shuffled_scenarios = pd.Series(unique_scenarios).sample(frac=1, random_state=1).tolist()  # Shuffle Scenario IDs
    
    # Reorder the dataframe by the shuffled Scenario IDs
    df_shuffled = pd.concat([df[df['Scenario ID'] == scenario] for scenario in shuffled_scenarios])
    
    # Reset index (optional)
    df_shuffled = df_shuffled.reset_index(drop=True)
    
    return df_shuffled


def split_train_test(df):
    # Define groups based on 'Scenario ID'
    groups = df['Scenario ID']
    
    # Create GroupShuffleSplit instance
    gss = sklearn.model_selection.GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=1)
    
    # Split the data
    for train_idx, test_idx in gss.split(df, groups=groups):
        return shuffle_df(df.iloc[train_idx]), shuffle_df(df.iloc[test_idx])

df_train, df_test = split_train_test(df_preprocessed)
show(df_train, 'df_train')
show(df_test, 'df_test')

,Scenario ID,Coordination strategy,Vehicle ID,Linearization C#0,Linearization C#1,Linearization C#2,Linearization C#3,Linearization C#4,Linearization C#5,Linearization C#6,...,Linearization C#94,Linearization C#95,Linearization C#96,Linearization C#97,Linearization C#98,Linearization C#99,"traveled total, m",No. of completed missions,No. of collisions,No. of near-misses
0,"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario7-5.json, stops, seed 1, probabilityForcingForHuman 1; AUTOMATED_FIRST",2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.746372,0.746372,0.746372,0.746372,0.82686,0.82686,75.5,0,0,0
1,"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario7-5.json, stops, seed 1, probabilityForcingForHuman 1; AUTOMATED_FIRST",2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,9096.5,11,0,0
2,"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario7-5.json, stops, seed 1, probabilityForcingForHuman 1; AUTOMATED_FIRST",2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,9460.0,6,0,0
3,"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario7-5.json, stops, seed 1, probabilityForcingForHuman 1; AUTOMATED_FIRST",2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,80.7,0,0,0
4,"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario7-5.json, stops, seed 1, probabilityForcingForHuman 1; AUTOMATED_FIRST",2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.746372,0.746372,0.746372,0.746372,0.82686,0.82686,75.5,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5731,"map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario1-7.json, stops, seed 1, probabilityForcingForHuman 1; AUTOMATED_FIRST",2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,7340.8,6,1,0
5732,"map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario1-7.json, stops, seed 1, probabilityForcingForHuman 1; AUTOMATED_FIRST",2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,9002.3,11,7,1
5733,"map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario1-7.json, stops, seed 1, probabilityForcingForHuman 1; AUTOMATED_FIRST",2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,8637.3,14,0,0
5734,"map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario1-7.json, stops, seed 1, probabilityForcingForHuman 1; AUTOMATED_FIRST",2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,8374.3,8,2,1


,Scenario ID,Coordination strategy,Vehicle ID,Linearization C#0,Linearization C#1,Linearization C#2,Linearization C#3,Linearization C#4,Linearization C#5,Linearization C#6,...,Linearization C#94,Linearization C#95,Linearization C#96,Linearization C#97,Linearization C#98,Linearization C#99,"traveled total, m",No. of completed missions,No. of collisions,No. of near-misses
0,"map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario8-6.json, baseline, seed 1, probabilityForcingForHuman 0; AUTOMATED_FIRST",0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3910.4,4,0,0
1,"map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario8-6.json, baseline, seed 1, probabilityForcingForHuman 0; AUTOMATED_FIRST",0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,8252.6,7,0,0
2,"map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario8-6.json, baseline, seed 1, probabilityForcingForHuman 0; AUTOMATED_FIRST",0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5514.5,8,0,0
3,"map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario8-6.json, baseline, seed 1, probabilityForcingForHuman 0; AUTOMATED_FIRST",0,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9415.9,5,0,0
4,"map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario8-6.json, baseline, seed 1, probabilityForcingForHuman 0; AUTOMATED_FIRST",0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3910.4,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,"map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario4-3.json, baseline, seed 1, probabilityForcingForHuman 0; AUTOMATED_FIRST",0,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9165.6,6,0,0
1436,"map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario4-3.json, baseline, seed 1, probabilityForcingForHuman 0; AUTOMATED_FIRST",0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6178.3,9,1,2
1437,"map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario4-3.json, baseline, seed 1, probabilityForcingForHuman 0; AUTOMATED_FIRST",0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9177.4,10,0,2
1438,"map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario4-3.json, baseline, seed 1, probabilityForcingForHuman 0; AUTOMATED_FIRST",0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9144.0,9,0,0


In [16]:
def split_df_to_X_y(df):
    columns_input_df = [col for col in df.columns
                        if col.split('#')[0] in COLUMNS_INPUT]
    columns_output_df = COLUMNS_OUTPUT
    assert set(COLUMNS_NONSPLIT + columns_input_df + columns_output_df) == set(df.columns)
    
    X = df[columns_input_df]
    y = df[columns_output_df]
    return X, y


def run_regression(df_train, df_test):
    X_train, y_train = split_df_to_X_y(df_train)
    X_test, y_test = split_df_to_X_y(df_test)
    
    model = sklearn.linear_model.LinearRegression()
    model.fit(X_train, y_train)
    ndarray_predictions = model.predict(X_test)
    df_predictions = pd.DataFrame(ndarray_predictions, columns=y_test.columns)
    return df_predictions


df_predictions_regression = run_regression(df_train, df_test)
df_predictions_regression

,"traveled total, m",No. of completed missions,No. of collisions,No. of near-misses
0,8308.870775,9.835980,4.943474,0.701534
1,7231.357911,4.451340,4.059048,0.042779
2,6577.721794,9.190561,-1.204134,-0.687006
3,8123.819144,9.695817,-0.330397,-0.373226
4,8308.870775,9.835980,4.943474,0.701534
...,...,...,...,...
1435,8837.657890,5.599062,0.026380,-0.206764
1436,8174.785760,12.009634,3.505805,0.589909
1437,8654.590690,10.712347,-0.148882,0.107685
1438,9067.410686,9.340648,-1.148468,-0.050175


In [17]:
def save_and_show(fig, basename):  # to avoid inlining large image data into the notebook file
    filename = f'{DIRECTORY_DATA}/{basename}-{random.random()}.png'
    fig.savefig(filename)
    
    # The `random` is because of https://stackoverflow.com/a/43640705.
    display(HTML(f'<img src="{filename}?{random.random()}" alt="{basename}" />'))
    
    plt.close(fig)
    
    return filename


def evaluate_and_plot_column(df_test, df_predictions, column):
    y_test_column = df_test[column]
    predictions_column = df_predictions[column]
    
    r2 = sklearn.metrics.r2_score(y_test_column, predictions_column)
    print(f"{column}:")
    print(f"- R^2 Score: {r2}")

    # Plot results for each output column
    fig = plt.figure(figsize=(10, 6))
    plt.scatter(y_test_column, predictions_column, color='blue', alpha=0.5)
    plt.plot([y_test_column.min(), y_test_column.max()], [y_test_column.min(), y_test_column.max()], 'k--', lw=2)
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.title(f'Actual vs Predicted Values for {column}')
    plt.grid(True)
    save_and_show(fig, f'Actual_vs_Predicted_Values_{column}')


def evaluate_and_plot_all_columns(df_test, df_predictions):
    for column in COLUMNS_OUTPUT:
        evaluate_and_plot_column(df_test, df_predictions, column)
        
        
evaluate_and_plot_all_columns(df_test, df_predictions_regression)

traveled total, m:
- R^2 Score: 0.3795419132969353


No. of completed missions:
- R^2 Score: 0.5260055492610368


No. of collisions:
- R^2 Score: 0.20676295199966566


No. of near-misses:
- R^2 Score: -0.5381978207289244


In [18]:
def run_autogluon(df_train, df_test):
    X_train, y_train = split_df_to_X_y(df_train)
    X_test, y_test = split_df_to_X_y(df_test)
    
    # Train AutoGluon model
    predictors = []
    for column in COLUMNS_OUTPUT:
        print(f'{column=}:')
        df_train_predictor = pd.concat([X_train, y_train[[column]]], axis=1)
        predictor = autogluon.tabular.TabularPredictor(
            label=column, 
            eval_metric='r2', 
            problem_type='regression',
        ).fit(
            df_train_predictor,
            presets='medium',
        )
        predictors.append(predictor)
    
    # Make predictions
    df_predictions = pd.DataFrame()
    for column, predictor in zip(COLUMNS_OUTPUT, predictors):
        df_predictions[column] = predictor.predict(X_test)

        # Leaderboard - Display a table of different models and their performance
        df_test_predictor = pd.concat([X_test, y_test[[column]]], axis=1)
        leaderboard = predictor.leaderboard(df_test_predictor, silent=True)
        show(leaderboard, f'Leaderboard for {column}')
        
    return df_predictions


df_predictions_autogluon = run_autogluon(df_train, df_test)
df_predictions_autogluon

No path specified. Models will be saved in: "AutogluonModels/ag-20241215_163955"
Preset alias specified: 'medium' maps to 'medium_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.12.2
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #135~20.04.1-Ubuntu SMP Mon Oct 7 13:56:22 UTC 2024
CPU Count:          16
Memory Avail:       6.13 GB / 31.09 GB (19.7%)
Disk Space Avail:   318.22 GB / 693.60 GB (45.9%)
Presets specified: ['medium']
Beginning AutoGluon training ...
AutoGluon will save models to "/home/olga/coordination_oru/scenario-analysis/AutogluonModels/ag-20241215_163955"
Train Data Rows:    5736
Train Data Columns: 102
Label Column:       traveled total, m
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    6278.12 MB
	Train Data (Origina

column='traveled total, m':


	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 100 | ['Linearization C#0', 'Linearization C#1', 'Linearization C#2', 'Linearization C#3', 'Linearization C#4', ...]
		('int', [])   :   2 | ['Coordination strategy', 'Vehicle ID']
	Types of features in processed data (raw dtype, special dtypes):
		('float', []) : 100 | ['Linearization C#0', 'Linearization C#1', 'Linearization C#2', 'Linearization C#3', 'Linearization C#4', ...]
		('int', [])   :   2 | ['Coordination strategy', 'Vehicle ID']
	0.2s = Fit runtime
	102 features in original data used to generate 102 features in processed data.
	Train Data (Processed) Memory Usage: 4.43 MB (0.1% of available memory)
Data preprocessing and feature engineering runtime = 0.22s ...
AutoGluon will gauge predictive performance using evaluation metric: 'r2'
	To change this, specify the eval_metric parameter of Predictor()
Automatically generating train/validation split with holdout_frac=0.1, Train Rows: 5162, Val Ro

[1000]	valid_set's l2: 1.57608e+06	valid_set's r2: 0.743504


	0.744	 = Validation score   (r2)
	5.7s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: LightGBM ...
/home/olga/miniconda3/lib/python3.12/site-packages/torch/cuda/__init__.py:716: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


...

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,NeuralNetTorch,0.846349,0.899943,r2,0.058108,0.063633,439.536206,0.058108,0.063633,439.536206,1,True,10
1,ExtraTreesMSE,0.832659,0.946607,r2,0.145620,0.120186,3.793191,0.145620,0.120186,3.793191,1,True,7
2,CatBoost,0.830695,0.946133,r2,0.021170,0.008753,72.421659,0.021170,0.008753,72.421659,1,True,6
3,RandomForestMSE,0.828575,0.944924,r2,0.206228,0.080405,8.080739,0.206228,0.080405,8.080739,1,True,5
4,NeuralNetFastAI,0.818736,0.837034,r2,0.026954,0.013400,10.946102,0.026954,0.013400,10.946102,1,True,8
5,LightGBM,0.818146,0.944964,r2,0.050249,0.020429,8.280445,0.050249,0.020429,8.280445,1,True,4
6,LightGBMLarge,0.817049,0.948564,r2,0.018218,0.004288,10.034420,0.018218,0.004288,10.034420,1,True,11
7,WeightedEnsemble_L2,0.815337,0.949522,r2,0.104179,0.079970,449.804748,0.002520,0.000705,0.163843,2,True,12
8,XGBoost,0.776917,0.944686,r2,0.037830,0.011984,12.364460,0.037830,0.011984,12.364460,1,True,9
9,KNeighborsUnif,0.601476,0.839329,r2,0.024602,0.018065,1.745706,0.024602,0.018065,1.745706,1,True,1


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost,0.916385,0.966366,r2,0.040451,0.003731,107.612424,0.040451,0.003731,107.612424,1,True,6
1,ExtraTreesMSE,0.904626,0.963430,r2,0.092608,0.034427,2.762621,0.092608,0.034427,2.762621,1,True,7
2,LightGBM,0.903125,0.966026,r2,0.145682,0.035728,12.745624,0.145682,0.035728,12.745624,1,True,4
3,WeightedEnsemble_L2,0.896441,0.968453,r2,0.111904,0.061531,113.793357,0.003118,0.000869,0.149901,2,True,12
4,NeuralNetFastAI,0.894514,0.908786,r2,0.030051,0.008541,3.912349,0.030051,0.008541,3.912349,1,True,8
5,XGBoost,0.887092,0.967236,r2,0.029539,0.007294,4.902986,0.029539,0.007294,4.902986,1,True,9
6,LightGBMLarge,0.878502,0.966306,r2,0.050592,0.013505,9.929192,0.050592,0.013505,9.929192,1,True,11
7,NeuralNetTorch,0.878038,0.929045,r2,0.059066,0.044653,108.715909,0.059066,0.044653,108.715909,1,True,10
8,RandomForestMSE,0.866333,0.963332,r2,0.103246,0.090487,6.752197,0.103246,0.090487,6.752197,1,True,5
9,LightGBMXT,0.855454,0.886407,r2,0.054596,0.013235,3.357192,0.054596,0.013235,3.357192,1,True,3


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,ExtraTreesMSE,0.640903,0.828979,r2,0.085196,0.033865,1.003129,0.085196,0.033865,1.003129,1,True,7
1,CatBoost,0.629318,0.838815,r2,0.013684,0.002550,13.747362,0.013684,0.002550,13.747362,1,True,6
2,RandomForestMSE,0.628926,0.832044,r2,0.092771,0.059048,5.440301,0.092771,0.059048,5.440301,1,True,5
3,WeightedEnsemble_L2,0.628175,0.843976,r2,0.146188,0.057100,151.100765,0.003461,0.000560,0.084288,2,True,12
4,LightGBM,0.621047,0.833564,r2,0.031962,0.009933,3.066550,0.031962,0.009933,3.066550,1,True,4
5,LightGBMLarge,0.620519,0.840019,r2,0.009937,0.001756,4.249841,0.009937,0.001756,4.249841,1,True,11
6,XGBoost,0.615956,0.835461,r2,0.022024,0.003901,2.689824,0.022024,0.003901,2.689824,1,True,9
7,NeuralNetFastAI,0.581578,0.792260,r2,0.032305,0.008457,4.335386,0.032305,0.008457,4.335386,1,True,8
8,LightGBMXT,0.547286,0.747722,r2,0.015258,0.004390,1.627545,0.015258,0.004390,1.627545,1,True,3
9,NeuralNetTorch,0.506471,0.765951,r2,0.060992,0.033982,128.653434,0.060992,0.033982,128.653434,1,True,10


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost,0.035763,0.613973,r2,0.011050,0.003530,5.545825,0.011050,0.003530,5.545825,1,True,6
1,NeuralNetTorch,-0.155701,-0.114544,r2,0.050922,0.046849,4.738155,0.050922,0.046849,4.738155,1,True,10
2,WeightedEnsemble_L2,-0.334750,0.624439,r2,0.110959,0.075265,15.419141,0.002073,0.000805,0.162330,2,True,12
3,LightGBMXT,-0.424113,0.575426,r2,0.012559,0.002852,1.043407,0.012559,0.002852,1.043407,1,True,3
4,NeuralNetFastAI,-0.568860,0.564633,r2,0.025728,0.011210,5.013912,0.025728,0.011210,5.013912,1,True,8
5,XGBoost,-0.610062,0.590654,r2,0.012861,0.004469,3.768705,0.012861,0.004469,3.768705,1,True,9
6,KNeighborsUnif,-0.643029,0.492624,r2,0.022122,0.008208,0.043931,0.022122,0.008208,0.043931,1,True,1
7,ExtraTreesMSE,-0.654426,0.574907,r2,0.067083,0.057603,1.570861,0.067083,0.057603,1.570861,1,True,7
8,KNeighborsDist,-0.766608,0.508051,r2,0.022862,0.008432,0.026166,0.022862,0.008432,0.026166,1,True,2
9,LightGBMLarge,-0.844622,0.608555,r2,0.005024,0.002118,3.126214,0.005024,0.002118,3.126214,1,True,11


,"traveled total, m",No. of completed missions,No. of collisions,No. of near-misses
0,6035.864258,6.947789,1.147731,0.001299
1,8382.136719,7.215522,0.408923,-0.001018
2,6256.576172,7.946746,-0.018323,0.020623
3,8817.482422,6.095346,0.256516,0.040046
4,6035.864258,6.947789,1.147731,0.001299
...,...,...,...,...
1435,9219.855469,6.106183,0.179387,0.017376
1436,4722.359375,6.814889,0.154102,0.519749
1437,9038.731445,9.374493,0.029001,0.205314
1438,8878.585938,9.034039,0.052910,0.011469


In [19]:
evaluate_and_plot_all_columns(df_test, df_predictions_autogluon)

traveled total, m:
- R^2 Score: 0.8153371297901711


No. of completed missions:
- R^2 Score: 0.8964411020278931


No. of collisions:
- R^2 Score: 0.6281751394271851


No. of near-misses:
- R^2 Score: -0.334749698638916
